In [1]:
%matplotlib inline
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import lib.lastfm as lastfm
import lib.curator as curator
from lib.spotify import User
import time
import calendar
import datetime
import re
import unicodedata
import importlib
import requests as http
import json
import uuid

In [2]:
token = 'BQCI41uJhXge_7gupELsPPgqmoX6egw1XbVQ35U5iIX9PF45HK8bsf5g5xHUxxE-_1tL7XLmKY3fsT0Xx409asnznWy_PHO_Cxw9cgMuIozRC61jAT83dLQXmbE_zs7S80vor7R1H76Yami2SfIi_hokDfjtU-LM64u0haulvK-FGqEkbN6qjXQSOMQtAxmnNDHaFQ6cUvSI'
received_features = ['danceability', 'energy', 'acousticness', 'valence', 'tempo']
test_user = User(received_features, 40, token=token)
user_lastfm = test_user.lastfm

In [3]:
winter = user_lastfm.get_count(test_user.library, (2016, 'winter'))
fall = user_lastfm.get_count(test_user.library, (2016, 'fall'))
summer = user_lastfm.get_count(test_user.library, (2016, 'summer'))
spring = user_lastfm.get_count(test_user.library, (2016, 'spring'))

In [4]:
pd.DataFrame({
        'track_name': test_user.library['track_name'],
        'winter': winter,
        'fall': fall,
        'summer': summer,
        'spring': spring
    })

,fall,spring,summer,track_name,winter
0,0,0,0,Wild for the Night,3
1,0,0,0,Electric Body,7
2,0,0,0,Lord Pretty Flacko Jodye 2 (LPFJ2),6
3,0,0,0,L$D,6
4,0,0,0,Canal St.,14
5,0,0,0,Come Get Her,5
6,0,0,0,All Day,4
7,0,0,0,6 Man,3
8,0,0,0,About the Money,5
9,0,0,0,Everyday,8


In [5]:
importlib.reload(curator)
curator.tag_filter(test_user.library, ['rap'])

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
0,Wild for the Night,2DP5IaZ0WEUhibImafxF1Y,"A$AP Rocky,Skrillex,Birdy Nam Nam,Lord Flacko",37,3,"hiphop,rap,dubstep,electronic,turntablism,elec...",0.750,0.856,0.053900,0.7310,139.973
1,Electric Body,2U7oXAJLPFNtAGYFYFLI7x,"A$AP Rocky,ScHoolboy Q",32,7,"hiphop,rap,hiphop,rap",0.904,0.590,0.428000,0.4780,120.006
2,Lord Pretty Flacko Jodye 2 (LPFJ2),77qHFaGdvYKoQtic8elZbz,A$AP Rocky,29,6,"hiphop,rap",0.692,0.672,0.009050,0.0638,103.982
3,L$D,4r28iBy4MUqkCpRjj7NU1W,A$AP Rocky,29,6,"hiphop,rap",0.560,0.249,0.209000,0.0371,120.125
4,Canal St.,00aF5EEoXdJreaknTVBoSI,"A$AP Rocky,Bones",30,14,"hiphop,rap,cloudrap,hiphop",0.687,0.592,0.338000,0.5380,136.996
5,Come Get Her,3sjxm3L7erH9sLUpMZzhd6,Rae Sremmurd,33,5,"hiphop,rap",0.688,0.560,0.162000,0.1170,185.025
6,All Day,4lFBXsmbUSebUBoNzE9XVr,"Kanye West,Theophilus London,Allan Kingdom,Pau...",38,4,"hiphop,rap,hiphop,electronic,hiphop,electronic...",0.829,0.630,0.036400,0.4710,123.004
7,6 Man,6I9CRrzpTrfkqo6KTfkpYZ,Drake,32,3,"hiphop,rap",0.632,0.546,0.239000,0.3540,83.114
8,About the Money,1Lphn7aWmLmrKIKDj5OEiD,"T.I.,Young Thug",32,5,"hiphop,rap,hiphop,rap",0.807,0.719,0.033100,0.5520,101.940
9,Everyday,3LsKaCwIuiWcPxTOSZApmE,"A$AP Rocky,Rod Stewart,Miguel,Mark Ronson",29,8,"hiphop,rap,classicrock,rock,rnb,soul,funk,british",0.395,0.656,0.009520,0.3180,149.830


In [6]:

curator.filter_by(test_user.library, 'count')

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
125,02:09,44nFlz3tVXesHRGuDvZfdD,EDEN,50,234,"ethereal,rock",0.347,0.246,0.984000,0.0304,159.969
528,Playground [feat. Bugle & Arama],592Ln8BTUH0wFK0jQZWZId,"Major Lazer,Bugle,Aramà",0,153,"seenlive,dancehall,dancehall,reggae",0.569,0.676,0.091700,0.7840,140.886
695,Cracks - Flux Pavilion Remix,4RUJy97KUspD5KOV8CO1GV,"Freestylers feat. Belle Humble,Freestylers,Bel...",18,139,"dubstep,,breakbeat,electronic",0.499,0.826,0.000950,0.4160,140.007
431,Give Me Everything,3QFbOgWOqXEtOD02FMcX7K,"Pitbull,Ne-Yo,Afrojack,Nayer",48,134,"hiphop,rap,rnb,soul,house,electrohouse,dance,pop",0.673,0.934,0.161000,0.4860,129.027
334,The Hills,4S73p2PcSonb5wbXIcanqv,The Weeknd,38,128,"rnb,electronic",0.587,0.565,0.064600,0.1400,112.908
770,Legacy,1wc971fXAVObXkpL8ZEqDT,Eminem,33,124,"rap,hiphop",0.673,0.898,0.240000,0.5870,164.170
452,Dum Dee Dum,5BRrBkpj0An7PViNqMCoGa,Keys N Krates,56,123,"trap,electronic",0.755,0.891,0.273000,0.5170,160.021
437,Freeway - Flux Pavilion and Kill The Noise Remix,2bdR4MKUcVfaZtu6Kv7BXt,Flux Pavilion,44,115,"dubstep,electronic",0.573,0.746,0.071800,0.1560,139.905
539,Five Hours,6zlOUIqcU6juXFww9UNpJK,Deorro,11,109,"house,dance",0.880,0.821,0.019700,0.5760,127.907
451,'Till I Collapse,6p4SUKhIyPqbmU828M0JWD,"Eminem,Nate Dogg",43,106,"rap,hiphop,gfunk,hiphop",0.559,0.840,0.070600,0.1130,171.311


In [7]:
curator.filter_by(test_user.library, 'count', reverse=True)

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
164,Die Young REMIX,75Liz40xlgtL7H9vsDF9tW,"Kesha,Juicy J,Wiz Khalifa,Becky G",40,0,"pop,dance,hiphop,rap,hiphop,rap,pop,femalevoca...",0.767,0.699,0.035500,0.6590,127.995
503,Higher Ground,0Js9hB2I66nJliPIJxzn2R,TNGHT (Hudson Mohawke x Lunice),55,0,"trap,electronic",0.711,0.761,0.598000,0.5260,160.738
175,Puzzles - Radio Edit,1sZnQFDHBNr3BRa89NnKo9,"Lenx & Denx,Andy Bianchini",47,0,"dance,electrohouse,trance",0.569,0.844,0.018900,0.2560,125.036
511,Just Can’t Get Enough,3JA9Jsuxr4xgHXEawAdCp4,The Black Eyed Peas,55,0,"hiphop,pop",0.483,0.648,0.187000,0.2380,93.939
161,Mirror,7GDvbciPx1FfglvYxYRYaR,"Lil Wayne,Bruno Mars",33,0,"rap,hiphop,pop,rnb",0.369,0.637,0.875000,0.3350,79.421
512,I Gotta Feeling,72np30cEzvqzxAUG5mrhyT,The Black Eyed Peas,53,0,"hiphop,pop",0.741,0.756,0.080300,0.6030,127.963
513,Boom Boom Pow,5MeZ3VxmtSv1O8PcekvLTn,The Black Eyed Peas,59,0,"hiphop,pop",0.868,0.848,0.151000,0.3450,130.115
524,Club Can't Handle Me (feat. David Guetta),0xcl9XT60Siji6CSG4y6nb,"Flo Rida,David Guetta",57,0,"hiphop,rap,house,dance",0.623,0.882,0.020500,0.5230,127.980
144,Heartbreak (Bare Noize Vocal Mix),2HFxvEw1uPkZ0vmfiNr9vZ,M'Black,20,0,"dubstep,downtempo",0.533,0.914,0.002340,0.2360,135.037
143,Keep It Mello (feat. Omar Lin X),0Q3SC6kEhxYagDP3bFe5K9,"Marshmello,Omar LinX",70,0,"trap,edm,hiphop,dubstep",0.686,0.888,0.001340,0.1140,142.015


In [8]:
curator.filter_by(test_user.library, 'popularity')

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
27,Now and Later,43jBqV3j3Xi1g6wO0bhIMd,Sage The Gemini,87,13,"hiphop,rap",0.781,0.570,0.273000,0.8580,107.059
176,Heathens,6i0V12jOa3mr6uu4WYhUBr,Twenty One Pilots,85,16,"electronic,indie",0.732,0.396,0.084100,0.5470,90.024
216,Don't Let Me Down,1i1fxkWeaMmKEB4T7zqbzK,"The Chainsmokers,Daya",85,14,"electronic,house,pop,american",0.532,0.869,0.157000,0.4190,159.803
29,Alone,3ujORpm8Hf7QWEtfqSAZL8,Alan Walker,81,4,"electronic,electronica",0.671,0.924,0.196000,0.1610,97.012
186,Alone,12mGwph2YzDIlChtq3EdXP,Marshmello,77,40,"trap,edm",0.636,0.953,0.024400,0.3970,142.011
259,My House,6Knv6wdA0luoMUuuoYi2i1,Flo Rida,77,9,"hiphop,rap",0.688,0.702,0.021500,0.7310,94.006
582,I'm Yours,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,77,60,"singersongwriter,acoustic",0.686,0.457,0.595000,0.7110,150.953
33,Losin Control,1jkIErXa3YNUX5QIyO6GGR,Russ,76,25,"rap,underlisteners",0.761,0.250,0.558000,0.2490,96.992
576,How to Save a Life,5fVZC9GiM4e8vu99W0Xf6J,The Fray,76,58,"alternative,rock",0.640,0.743,0.269000,0.3560,122.035
557,In The End,60a0Rd6pjrkxjPbaKzXjfq,Linkin Park,76,79,"rock,numetal",0.553,0.900,0.011300,0.3970,105.213


In [9]:
curator.filter_by(test_user.library, 'danceability')

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
748,Around The World,1pKYYY0dkg23sQQXi0Q5zN,Daft Punk,63,50,"electronic,dance",0.956,0.795,0.003560,0.8400,121.294
325,Without Me,7n8eZAUj6vevNQZ0EF3OH0,Eminem,25,33,"rap,hiphop",0.926,0.626,0.003010,0.6630,112.246
81,Fake Love,2EDLxm55ESkYQ1h0Isfwdv,Drake,44,19,"hiphop,rap",0.924,0.526,0.081600,0.6550,133.972
525,Low (feat T-Pain),0CAfXk7DXMnon4gLudAp7J,"Flo Rida,T-Pain",70,10,"hiphop,rap,rnb,hiphop",0.918,0.609,0.092800,0.3040,128.008
604,By Your Side,7lXCT5xxTVOJAu11FOF8PS,Big Chocolate,0,69,"dubstep,electronic",0.914,0.624,0.086600,0.6260,150.034
324,Cleanin' Out My Closet,2Vyny7HMXDxIw4NqqkViAn,Eminem,26,30,"rap,hiphop",0.907,0.756,0.081600,0.9010,148.017
1,Electric Body,2U7oXAJLPFNtAGYFYFLI7x,"A$AP Rocky,ScHoolboy Q",32,7,"hiphop,rap,hiphop,rap",0.904,0.590,0.428000,0.4780,120.006
223,Moar Ghosts 'n' Stuff,2lXzgU19U9cSZLJtawjob6,deadmau5,39,23,"house,electronic",0.903,0.648,0.000105,0.3550,128.014
258,Drop It Like It's Hot - Album Version (Edited),18UhvUi9FjHtoTgnqX7kbl,"Snoop Dogg,Pharrell Williams",28,11,"hiphop,rap,hiphop,rnb",0.903,0.550,0.217000,0.6470,92.009
331,Hotline Bling,6nmz4imkDcmtwMjocAzFSx,Drake,18,38,"hiphop,rap",0.896,0.623,0.002570,0.5640,134.962


In [10]:
curator.filter_in_range(test_user.library, 'tempo', 120, 130)

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
1,Electric Body,2U7oXAJLPFNtAGYFYFLI7x,"A$AP Rocky,ScHoolboy Q",32,7,"hiphop,rap,hiphop,rap",0.904,0.590,0.428000,0.4780,120.006
3,L$D,4r28iBy4MUqkCpRjj7NU1W,A$AP Rocky,29,6,"hiphop,rap",0.560,0.249,0.209000,0.0371,120.125
6,All Day,4lFBXsmbUSebUBoNzE9XVr,"Kanye West,Theophilus London,Allan Kingdom,Pau...",38,4,"hiphop,rap,hiphop,electronic,hiphop,electronic...",0.829,0.630,0.036400,0.4710,123.004
12,OWM,3wwmrdvgaHPnwKxJ4wCnW6,GITCHII,24,13,,0.488,0.774,0.000386,0.0478,129.820
16,Beautiful Girls,5EuAj9nZiWRKeBkgxMXsW4,Sean Kingston,14,22,"rnb,hiphop",0.745,0.666,0.159000,0.7270,129.995
20,Stellar - Morgan Page Remix,5eleUvVKuyL8mvOpkEIS2q,"Disco Killerz,Liquid Todd,Jimmy Gnecco,Morgan ...",42,1,",,house,electronica,singersongwriter,rock,hous...",0.520,0.923,0.042700,0.1840,128.004
23,Beneath with Me (feat. Skylar Grey) - Kaskade'...,3XNhUO3VysNZUGq3Z16SWZ,"Kaskade,deadmau5,Skylar Grey",56,9,"house,electronic,house,electronic,pop,alternative",0.518,0.525,0.102000,0.1710,125.793
25,Bambini,3p3XKoz87Ihxhqw6Uj9fr9,Mike Williams,63,7,deepsoul,0.477,0.916,0.000763,0.1080,127.990
38,Miracle,7p9jHBSk1pOciORji5sPV8,"Borgeous,BRKLYN,Lenachka",42,14,"electronic,house,electronic,,pop,indie",0.468,0.776,0.006400,0.2170,127.982
41,Tsunami - Original Mix,3felG0BIvmmOoCIeE2A5vq,"DVBBS,Borgeous",53,6,"house,electronic,electronic,house",0.698,0.838,0.049000,0.2890,128.004


In [11]:
# pass a dictionary with format characteristic: feature: params or None
filters = {
    'count': None,
    'tags': None,
    'timeslice': None,
    'popularity': None,
    'danceability': ['filter_by'],
    'energy': ['filter_by'],
    'acousticness': None,
    'valence': None,
    'tempo': None
}

importlib.reload(curator)
importlib.reload(lastfm)
test_df = curator.filter_with(test_user, filters)

In [12]:
len(test_df)

108

In [13]:
def foo(a, b='what'):
    print(a, b)
    
params = {}
foo(1)
foo(1, **params)

1 what
1 what


In [14]:
request_body = {
    'filters': filters,
    'uid': 'bornofawesomeness'
}
api_url = 'http://127.0.0.1:5000/create'
data = json.dumps(request_body)
response = http.post(api_url, data=data, headers={'Content-Type': 'application/json'})

In [15]:
response_json = response.json()
playlist_id = response_json['identifier']
playlist = pd.read_json(response_json['playlist'], orient='split')
playlist

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
0,Wild for the Night,2DP5IaZ0WEUhibImafxF1Y,"A$AP Rocky,Skrillex,Birdy Nam Nam,Lord Flacko",37,0,"hiphop,rap,dubstep,electronic,turntablism,elec...",0.750,0.856,0.053900,0.7310,139.973
1,Alone,3ujORpm8Hf7QWEtfqSAZL8,Alan Walker,81,0,"electronic,electronica",0.671,0.924,0.196000,0.1610,97.012
2,Febreze (feat. 2 Chainz),48bSfSZaq9Aizbu4AWn4st,"Jack Ü,Skrillex,Diplo,2 Chainz",60,0,"electronic,trap,dubstep,electronic,electronic,...",0.680,0.983,0.018400,0.2920,149.829
3,Enigma (feat. GRRL PAL),5wd14CMjlqbSjibnpoTgla,"SLUMBERJACK,GRRL PAL",52,0,"electronic,chillout,pop,electronic",0.631,0.909,0.006830,0.6360,153.931
4,Rebel,0htTZnlk6okQ1HIq4EvFQ6,G-Eazy,50,1,"hiphop,rap",0.682,0.911,0.143000,0.5540,140.079
5,Ni**as In Paris,4Li2WHPkuyCdtmokzW2007,"JAY Z,Kanye West",72,0,"hiphop,rap,hiphop,rap",0.788,0.859,0.127000,0.7650,140.027
6,Dum Dee Dum - JiKay Remix,2zUwVdcaXijMUqLsp9BycM,Keys N Krates,47,0,"trap,electronic",0.781,0.854,0.015900,0.5010,100.043
7,Cutting Shapes,3DDBSKvXiGEN4MiuqbJq34,Don Diablo,66,0,"house,seenlive",0.712,0.913,0.024900,0.5650,125.901
8,Ultra DJ's/Me & You - Spencer & Hill Remix,20qsOjwLKHgY28t6CNTbMq,Dj Suraci And Spins,22,0,,0.667,0.927,0.000346,0.6660,129.990
9,Yallah,23t5Og9w3EQvMWezXmUT5H,UZ,41,0,"trap,electronic",0.791,0.928,0.011000,0.5290,144.954


In [21]:
save_data = {
    'identifier': playlist_id,
    'uid': 'bornofawesomeness',
    'name': 'litty lit lit'
}
api_url = 'http://127.0.0.1:5000/save'
save_data = json.dumps(save_data)
response = http.post(api_url, data=save_data, headers={'Content-Type': 'application/json'})
response.text

'awesome'

In [17]:
a = pd.DataFrame([
        [1, 2, 3],
        [4, 5, 6], 
        [0, 9, 8]
    ])
b = pd.DataFrame([
        [1, 2, 3],
        [0, 0, 0],
        [4, 5, 5]
    ])
pd.merge(a, b)

,0,1,2
0,1,2,3


In [18]:
energy = curator.filter_by(test_user.library, 'energy')
dance = curator.filter_by(test_user.library, 'danceability')
energy

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
631,Satisfaction - RL Grime Remix,68J8vrLOTrUUOzaPLtWOHi,"Benny Benassi,The Biz",38,53,"electronic,house,house,electronic",0.783,0.999,0.129000,0.1250,145.020
135,TKO,6JfmjCGy7nYztlgmWrXdsE,"Bassnectar,Rye Rye,Zion I",50,25,"dubstep,electronic,hiphop,electronic,hiphop,un...",0.544,0.995,0.005450,0.4630,87.999
496,Run Riot - Sub Zero Remix,5TtPEFYnYlzEbEsA5nHJYq,Camo & Krooked,0,34,"drumandbass,dubstep",0.463,0.992,0.000221,0.1850,175.050
449,Sandstorm - Radio Edit,3XWZ7PNB3ei50bTPzHhqA6,Darude,49,56,"trance,electronic",0.494,0.990,0.093600,0.6450,136.363
68,American Idiot,6nTiIhLmQ3FWhvrGafw2zj,Green Day,71,15,"punkrock,rock",0.380,0.988,0.000026,0.7770,186.113
640,Superior,7tf63j8xFrQz0hwtWsxVfA,PANTyRAiD,17,64,"dubstep,glitch",0.592,0.985,0.133000,0.0873,140.032
676,Big Boss,34o2xJAVNTzoYN0o2Sd6sW,Doctor P,38,29,"dubstep,electronic",0.553,0.983,0.000021,0.0272,139.932
605,Cinema - Skrillex Remix,6jNM6fKx3WnyHbWBI9hs2r,"Benny Benassi,Gary Go",57,0,"electronic,house,pop,british",0.635,0.983,0.009010,0.4220,145.085
32,Febreze (feat. 2 Chainz),48bSfSZaq9Aizbu4AWn4st,"Jack Ü,Skrillex,Diplo,2 Chainz",60,13,"electronic,trap,dubstep,electronic,electronic,...",0.680,0.983,0.018400,0.2920,149.829
545,Red Step,1GcaJ6MZs2UzWlhDCFNbMD,"Bassnectar,Jantsen",27,5,"dubstep,electronic,dubstep,glitchhop",0.686,0.980,0.000179,0.1800,140.007


In [19]:
dance

,track_name,track_id,artists,popularity,count,genres,danceability,energy,acousticness,valence,tempo
748,Around The World,1pKYYY0dkg23sQQXi0Q5zN,Daft Punk,63,50,"electronic,dance",0.956,0.795,0.003560,0.8400,121.294
325,Without Me,7n8eZAUj6vevNQZ0EF3OH0,Eminem,25,33,"rap,hiphop",0.926,0.626,0.003010,0.6630,112.246
81,Fake Love,2EDLxm55ESkYQ1h0Isfwdv,Drake,44,19,"hiphop,rap",0.924,0.526,0.081600,0.6550,133.972
525,Low (feat T-Pain),0CAfXk7DXMnon4gLudAp7J,"Flo Rida,T-Pain",70,10,"hiphop,rap,rnb,hiphop",0.918,0.609,0.092800,0.3040,128.008
604,By Your Side,7lXCT5xxTVOJAu11FOF8PS,Big Chocolate,0,69,"dubstep,electronic",0.914,0.624,0.086600,0.6260,150.034
324,Cleanin' Out My Closet,2Vyny7HMXDxIw4NqqkViAn,Eminem,26,30,"rap,hiphop",0.907,0.756,0.081600,0.9010,148.017
1,Electric Body,2U7oXAJLPFNtAGYFYFLI7x,"A$AP Rocky,ScHoolboy Q",32,7,"hiphop,rap,hiphop,rap",0.904,0.590,0.428000,0.4780,120.006
223,Moar Ghosts 'n' Stuff,2lXzgU19U9cSZLJtawjob6,deadmau5,39,23,"house,electronic",0.903,0.648,0.000105,0.3550,128.014
258,Drop It Like It's Hot - Album Version (Edited),18UhvUi9FjHtoTgnqX7kbl,"Snoop Dogg,Pharrell Williams",28,11,"hiphop,rap,hiphop,rnb",0.903,0.550,0.217000,0.6470,92.009
331,Hotline Bling,6nmz4imkDcmtwMjocAzFSx,Drake,18,38,"hiphop,rap",0.896,0.623,0.002570,0.5640,134.962


In [20]:
count = 0
for energy_song in energy.itertuples():
    for dance_song in dance.itertuples():
        if energy_song == dance_song:
            count += 1
count

108